# Model MT5
attention based model
text -> Model -> new text

In [66]:
import tensorflow
import os
import pandas as pd



In [67]:
path = 'training'
os.listdir(path)

['europarl-v7.cs-en.cs',
 'europarl-v7.cs-en.en',
 'europarl-v7.de-en.de',
 'europarl-v7.de-en.en',
 'europarl-v7.es-en.en',
 'europarl-v7.es-en.es',
 'europarl-v7.fr-en.en',
 'europarl-v7.fr-en.fr',
 'questions_easy.csv']

In [68]:
with open(path+'/europarl-v7.de-en.en',encoding='utf-8') as f: en = f.read().split('\n')

In [69]:
with open(path+'/europarl-v7.de-en.de',encoding='utf-8') as f: de = f.read().split('\n')

In [70]:
en[0]

'Resumption of the session'

In [71]:
de[0]

'Wiederaufnahme der Sitzungsperiode'

In [72]:
import re

In [73]:
re_eq = re.compile('^(Wh[^?.!]+\?)')
re_dq = re.compile('^([^?.!]+\?)')
en_fname = path+'/europarl-v7.de-en.en'
de_fname = path+'/europarl-v7.de-en.de'

In [74]:
lines = ((eq,fq)
for eq, fq in zip(open(en_fname, encoding='utf-8'), open(de_fname, encoding='utf-8')))
qs = [(e, d) for e,d in lines if e and d]

In [75]:
qs = qs[:100000]

In [76]:
len(qs)

100000

In [77]:
import numpy as np

In [78]:
qs = np.array(qs)

In [79]:
qs[:,0]

array(['Resumption of the session\n',
       'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.\n',
       "Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.\n",
       ...,
       'Freight is growing out of all proportion too, and is expected to increase by 70% over the next ten years.\n',
       'It is surprising that there is no breakthrough in intermodal transport in the offing.\n',
       'Perhaps the pressure on the transporters and the shippers is not yet so great as to bring about this breakthrough.\n'],
      dtype='<U1872')

In [80]:
mark_start = 'ssss '
mark_end = ' eeee'
for index, word in enumerate(qs[:,0]):
    qs[:,0][index] = word[:-1]
    #qs[:,0][index] = mark_start + word + mark_end
for index, word in enumerate(qs[:,1]):
    qs[:,1][index] = word[:-1]  

In [81]:
qs = [(q1,q2) for q1,q2 in qs]
df = pd.DataFrame({'de': [q[1] for q in qs], 'en': [q[0] for q in qs]}, columns = ['en', 'de'])
df.to_csv(path+'/questions_easy.csv', index=False)

In [82]:
qs = np.array(qs)

In [83]:
qs[1]

array(['I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.',
       'Ich erkläre die am Freitag, dem 17. Dezember unterbrochene Sitzungsperiode des Europäischen Parlaments für wiederaufgenommen, wünsche Ihnen nochmals alles Gute zum Jahreswechsel und hoffe, daß Sie schöne Ferien hatten.'],
      dtype='<U1871')

In [84]:
for index, word in enumerate(qs[:,0]):
    #qs[:,0][index] = word[:-1]
    qs[:,0][index] = mark_start + word + mark_end

In [85]:
qs[0]

array(['ssss Resumption of the session eeee',
       'Wiederaufnahme der Sitzungsperiode'], dtype='<U1871')

In [86]:
df = pd.read_csv(path+'/questions_easy.csv')
df.head()

,en,de
0,Resumption of the session,Wiederaufnahme der Sitzungsperiode
1,I declare resumed the session of the European ...,"Ich erkläre die am Freitag, dem 17. Dezember u..."
2,"Although, as you will have seen, the dreaded '...","Wie Sie feststellen konnten, ist der gefürchte..."
3,You have requested a debate on this subject in...,Im Parlament besteht der Wunsch nach einer Aus...
4,"In the meantime, I should like to observe a mi...",Heute möchte ich Sie bitten - das ist auch der...


# Preprocessing

In [87]:
df.dtypes

en    object
de    object
dtype: object

In [88]:
df = df.astype(str)

In [89]:
df['en'] = df['en'].apply(lambda x:x.lower())
df['de'] = df['de'].apply(lambda x:x.lower())

In [90]:
df.head()

,en,de
0,resumption of the session,wiederaufnahme der sitzungsperiode
1,i declare resumed the session of the european ...,"ich erkläre die am freitag, dem 17. dezember u..."
2,"although, as you will have seen, the dreaded '...","wie sie feststellen konnten, ist der gefürchte..."
3,you have requested a debate on this subject in...,im parlament besteht der wunsch nach einer aus...
4,"in the meantime, i should like to observe a mi...",heute möchte ich sie bitten - das ist auch der...


In [91]:
from tqdm.notebook import tqdm

In [92]:
import numpy as np

In [93]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [94]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
class TokenizerWrap(Tokenizer):
    """Wrap the Tokenizer-class from Keras with more functionality."""
    
    def __init__(self, texts, padding,
                 reverse=False, num_words=None):
        """
        :param texts: List of strings. This is the data-set.
        :param padding: Either 'post' or 'pre' padding.
        :param reverse: Boolean whether to reverse token-lists.
        :param num_words: Max number of words to use.
        """

        Tokenizer.__init__(self, num_words=num_words)

        # Create the vocabulary from the texts.
        self.fit_on_texts(texts)

        # Create inverse lookup from integer-tokens to words.
        self.index_to_word = dict(zip(self.word_index.values(),
                                      self.word_index.keys()))

        # Convert all texts to lists of integer-tokens.
        # Note that the sequences may have different lengths.
        self.tokens = self.texts_to_sequences(texts)

        if reverse:
            # Reverse the token-sequences.
            self.tokens = [list(reversed(x)) for x in self.tokens]
        
            # Sequences that are too long should now be truncated
            # at the beginning, which corresponds to the end of
            # the original sequences.
            truncating = 'pre'
        else:
            # Sequences that are too long should be truncated
            # at the end.
            truncating = 'post'

        # The number of integer-tokens in each sequence.
        self.num_tokens = [len(x) for x in self.tokens]

        # Max number of tokens to use in all sequences.
        # We will pad / truncate all sequences to this length.
        # This is a compromise so we save a lot of memory and
        # only have to truncate maybe 5% of all the sequences.
        self.max_tokens = np.mean(self.num_tokens) \
                          + 2 * np.std(self.num_tokens)
        self.max_tokens = int(self.max_tokens)

        # Pad / truncate all token-sequences to the given length.
        # This creates a 2-dim numpy matrix that is easier to use.
        self.tokens_padded = pad_sequences(self.tokens,
                                           maxlen=self.max_tokens,
                                           padding=padding,
                                           truncating=truncating)

    def token_to_word(self, token):
        """Lookup a single word from an integer-token."""

        word = " " if token == 0 else self.index_to_word[token]
        return word 

    def tokens_to_string(self, tokens):
        """Convert a list of integer-tokens to a string."""

        # Create a list of the individual words.
        words = [self.index_to_word[token]
                 for token in tokens
                 if token != 0]
        
        # Concatenate the words to a single string
        # with space between all the words.
        text = " ".join(words)

        return text
    
    def text_to_tokens(self, text, reverse=False, padding=False):
        """
        Convert a single text-string to tokens with optional
        reversal and padding.
        """

        # Convert to tokens. Note that we assume there is only
        # a single text-string so we wrap it in a list.
        tokens = self.texts_to_sequences([text])
        tokens = np.array(tokens)

        if reverse:
            # Reverse the tokens.
            tokens = np.flip(tokens, axis=1)

            # Sequences that are too long should now be truncated
            # at the beginning, which corresponds to the end of
            # the original sequences.
            truncating = 'pre'
        else:
            # Sequences that are too long should be truncated
            # at the end.
            truncating = 'post'

        if padding:
            # Pad and truncate sequences to the given length.
            tokens = pad_sequences(tokens,
                                   maxlen=self.max_tokens,
                                   padding='pre',
                                   truncating=truncating)

        return tokens

In [95]:
num_words =10000
tokenizer_src = TokenizerWrap(texts=qs[:,0],
                              padding='pre',
                              reverse=True,
                              num_words=num_words)

In [96]:
tokenizer_dest = TokenizerWrap(texts=qs[:,1],
                               padding='post',
                               reverse=False,
                               num_words=num_words)

In [97]:
tokens_src = tokenizer_src.tokens_padded
tokens_dest = tokenizer_dest.tokens_padded
print(tokens_src.shape)
print(tokens_dest.shape)

(100000, 57)
(100000, 48)


In [98]:
tokens_src[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    3,  926,    1,    4,
       5176,    2])

In [99]:
token_start = tokenizer_src.word_index[mark_end.strip()]
token_start

3

In [100]:
token_end = tokenizer_dest.word_index['weshalb']
token_end

978

In [101]:
idx = 2

In [102]:
tokens_src[idx]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    3, 4987,  106, 1610,   10, 1823,
        954,    4, 1450,    9, 2139,   68,    4,  207,    9,    7,   82,
          1,  180, 7944,    5, 1710,    1,  639,   22,   26,   44,   20,
        362,    2])

In [103]:
tokenizer_src.tokens_to_string(tokens_src[idx])

'eeee dreadful were truly that disasters natural of series a suffered countries of number a in people the still materialise to failed the seen have will you as although ssss'

In [104]:
qs[:,0][idx]

"ssss Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful. eeee"

In [105]:
encoder_input_data = tokens_src
decoder_input_data = tokens_dest[:, :-1]
decoder_output_data = tokens_dest[:, 1:]
decoder_output_data.shape

(100000, 47)

In [106]:
tokens_dest[idx]

array([  30,   23,  755, 1034,   12,    2,   14, 4648,  100,   28,  150,
        923,  123,   76,  634,   10, 4326, 4250,  955,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0])

In [107]:
tokenizer_dest.tokens_to_string(tokens_dest[idx])

'wie sie feststellen konnten ist der nicht eingetreten doch sind bürger einiger unserer mitgliedstaaten opfer von schrecklichen naturkatastrophen geworden'

In [108]:
qs[:,1][idx]


'Wie Sie feststellen konnten, ist der gefürchtete "Millenium-Bug " nicht eingetreten. Doch sind Bürger einiger unserer Mitgliedstaaten Opfer von schrecklichen Naturkatastrophen geworden.'

In [109]:
from tensorflow.keras.layers import *

In [110]:
encoder_input = Input(shape=(None, ), name='encoder_input')

In [111]:
embedding_size = 128

In [112]:
encoder_embedding = Embedding(input_dim=num_words,
                              output_dim=embedding_size,
                              name='encoder_embedding')

In [113]:
state_size = 512

In [114]:
encoder_gru1 = GRU(state_size, name='encoder_gru1',
                   return_sequences=True)
encoder_gru2 = GRU(state_size, name='encoder_gru2',
                   return_sequences=True)
encoder_gru3 = GRU(state_size, name='encoder_gru3',
                   return_sequences=False)

In [115]:
def connect_encoder():
    # Start the neural network with its input-layer.
    net = encoder_input
    
    # Connect the embedding-layer.
    net = encoder_embedding(net)

    # Connect all the GRU-layers.
    net = encoder_gru1(net)
    net = encoder_gru2(net)
    net = encoder_gru3(net)

    # This is the output of the encoder.
    encoder_output = net
    
    return encoder_output

In [116]:
encoder_output = connect_encoder()

In [117]:
decoder_initial_state = Input(shape=(state_size,),
                              name='decoder_initial_state')

In [118]:
decoder_input = Input(shape=(None, ), name='decoder_input')

In [119]:
decoder_embedding = Embedding(input_dim=num_words,
                              output_dim=embedding_size,
                              name='decoder_embedding')

In [120]:
decoder_gru1 = GRU(state_size, name='decoder_gru1',
                   return_sequences=True)
decoder_gru2 = GRU(state_size, name='decoder_gru2',
                   return_sequences=True)
decoder_gru3 = GRU(state_size, name='decoder_gru3',
                   return_sequences=True)

In [121]:
decoder_dense = Dense(num_words,
                      activation='softmax',
                      name='decoder_output')

In [122]:
def connect_decoder(initial_state):
    # Start the decoder-network with its input-layer.
    net = decoder_input

    # Connect the embedding-layer.
    net = decoder_embedding(net)
    
    # Connect all the GRU-layers.
    net = decoder_gru1(net, initial_state=initial_state)
    net = decoder_gru2(net, initial_state=initial_state)
    net = decoder_gru3(net, initial_state=initial_state)

    # Connect the final dense layer that converts to
    # one-hot encoded arrays.
    decoder_output = decoder_dense(net)
    
    return decoder_output

In [123]:
from tensorflow.keras.models import Model

In [124]:
decoder_output = connect_decoder(initial_state=encoder_output)

model_train = Model(inputs=[encoder_input, decoder_input],
                    outputs=[decoder_output])

In [125]:
model_encoder = Model(inputs=[encoder_input],
                      outputs=[encoder_output])

In [126]:
decoder_output = connect_decoder(initial_state=decoder_initial_state)

model_decoder = Model(inputs=[decoder_input, decoder_initial_state],
                      outputs=[decoder_output])

In [127]:
from tensorflow.keras.optimizers import *

In [128]:
model_train.compile(optimizer=RMSprop(learning_rate=1e-3),
                    loss='sparse_categorical_crossentropy')

In [129]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard


In [130]:
path_checkpoint = '21_checkpoint.keras'
callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint,
                                      monitor='val_loss',
                                      verbose=1,
                                      save_weights_only=True,
                                      save_best_only=True)

In [131]:
callback_early_stopping = EarlyStopping(monitor='val_loss',
                                        patience=100, verbose=1)

In [132]:
callback_tensorboard = TensorBoard(log_dir='./21_logs/',
                                   histogram_freq=0,
                                   write_graph=False)

In [133]:
callbacks = [callback_early_stopping,
             callback_checkpoint,
             callback_tensorboard]

In [134]:
try:
    model_train.load_weights(path_checkpoint)
except Exception as error:
    print("Error trying to load checkpoint.")
    print(error)

In [389]:
x_data = \
{
    'encoder_input': encoder_input_data,
    'decoder_input': decoder_input_data
}

In [390]:
y_data = \
{
    'decoder_output': decoder_output_data
}

In [391]:
validation_split = 10000 / len(encoder_input_data)
validation_split

0.1

In [392]:
model_train.fit(x=x_data,
                y=y_data,
                batch_size=128,
                epochs=1000,
                validation_split=validation_split,
                callbacks=callbacks)

Epoch 1/1000
704/704 [==============================] - ETA: 0s - loss: 2.5578
Epoch 1: val_loss improved from inf to 2.49781, saving model to 21_checkpoint.keras
704/704 [==============================] - 150s 204ms/step - loss: 2.5578 - val_loss: 2.4978
Epoch 2/1000
703/704 [============================>.] - ETA: 0s - loss: 2.2076
Epoch 2: val_loss improved from 2.49781 to 2.33785, saving model to 21_checkpoint.keras
704/704 [==============================] - 144s 204ms/step - loss: 2.2076 - val_loss: 2.3378
Epoch 3/1000
703/704 [============================>.] - ETA: 0s - loss: 2.0469
Epoch 3: val_loss improved from 2.33785 to 2.25086, saving model to 21_checkpoint.keras
704/704 [==============================] - 143s 204ms/step - loss: 2.0468 - val_loss: 2.2509
Epoch 4/1000
703/704 [============================>.] - ETA: 0s - loss: 1.9390
Epoch 4: val_loss improved from 2.25086 to 2.20979, saving model to 21_checkpoint.keras
704/704 [==============================] - 144s 205ms/ste

Epoch 37/1000
703/704 [============================>.] - ETA: 0s - loss: 0.9763
Epoch 37: val_loss did not improve from 2.16432
704/704 [==============================] - 142s 202ms/step - loss: 0.9764 - val_loss: 2.5075
Epoch 38/1000
703/704 [============================>.] - ETA: 0s - loss: 0.9649
Epoch 38: val_loss did not improve from 2.16432
704/704 [==============================] - 142s 202ms/step - loss: 0.9649 - val_loss: 2.5201
Epoch 39/1000
703/704 [============================>.] - ETA: 0s - loss: 0.9547
Epoch 39: val_loss did not improve from 2.16432
704/704 [==============================] - 142s 202ms/step - loss: 0.9548 - val_loss: 2.5287
Epoch 40/1000
703/704 [============================>.] - ETA: 0s - loss: 0.9452
Epoch 40: val_loss did not improve from 2.16432
704/704 [==============================] - 142s 202ms/step - loss: 0.9453 - val_loss: 2.5418
Epoch 41/1000
703/704 [============================>.] - ETA: 0s - loss: 0.9356
Epoch 41: val_loss did not improve f

In [135]:
def translate(input_text, true_output_text=None):
    """Translate a single text-string."""

    # Convert the input-text to integer-tokens.
    # Note the sequence of tokens has to be reversed.
    # Padding is probably not necessary.
    input_tokens = tokenizer_src.text_to_tokens(text=input_text,
                                                reverse=True,
                                                padding=True)
    
    # Get the output of the encoder's GRU which will be
    # used as the initial state in the decoder's GRU.
    # This could also have been the encoder's final state
    # but that is really only necessary if the encoder
    # and decoder use the LSTM instead of GRU because
    # the LSTM has two internal states.
    initial_state = model_encoder.predict(input_tokens)

    # Max number of tokens / words in the output sequence.
    max_tokens = tokenizer_dest.max_tokens

    # Pre-allocate the 2-dim array used as input to the decoder.
    # This holds just a single sequence of integer-tokens,
    # but the decoder-model expects a batch of sequences.
    shape = (1, max_tokens)
    decoder_input_data = np.zeros(shape=shape, dtype=np.int)

    # The first input-token is the special start-token for 'ssss '.
    token_int = token_start

    # Initialize an empty output-text.
    output_text = ''

    # Initialize the number of tokens we have processed.
    count_tokens = 0

    # While we haven't sampled the special end-token for ' eeee'
    # and we haven't processed the max number of tokens.
    while token_int != token_end and count_tokens < max_tokens:
        # Update the input-sequence to the decoder
        # with the last token that was sampled.
        # In the first iteration this will set the
        # first element to the start-token.
        decoder_input_data[0, count_tokens] = token_int

        # Wrap the input-data in a dict for clarity and safety,
        # so we are sure we input the data in the right order.
        x_data = \
        {
            'decoder_initial_state': initial_state,
            'decoder_input': decoder_input_data
        }

        # Note that we input the entire sequence of tokens
        # to the decoder. This wastes a lot of computation
        # because we are only interested in the last input
        # and output. We could modify the code to return
        # the GRU-states when calling predict() and then
        # feeding these GRU-states as well the next time
        # we call predict(), but it would make the code
        # much more complicated.

        # Input this data to the decoder and get the predicted output.
        decoder_output = model_decoder.predict(x_data)

        # Get the last predicted token as a one-hot encoded array.
        token_onehot = decoder_output[0, count_tokens, :]
        
        # Convert to an integer-token.
        token_int = np.argmax(token_onehot)

        # Lookup the word corresponding to this integer-token.
        sampled_word = tokenizer_dest.token_to_word(token_int)

        # Append the word to the output-text.
        output_text += " " + sampled_word

        # Increment the token-counter.
        count_tokens += 1

    # Sequence of tokens output by the decoder.
    output_tokens = decoder_input_data[0]
    
    # Print the input-text.
    print("Input text:")
    print(input_text)
    print()

    # Print the translated output-text.
    print("Translated text:")
    print(output_text)
    print()

    # Optionally print the true translated text.
    if true_output_text is not None:
        print("True output text:")
        print(true_output_text)
        print()

In [136]:
idx = 3
translate(input_text=qs[:,0][idx],
          true_output_text=qs[:,1][idx])

C:\Users\Tammaa\anaconda3\envs\env1\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Input text:
ssss You have requested a debate on this subject in the course of the next few days, during this part-session. eeee

Translated text:
 in einer zeit in diesem plenum ist ein thema in der konferenz in der region                                                                  

True output text:
Im Parlament besteht der Wunsch nach einer Aussprache im Verlauf dieser Sitzungsperiode in den nächsten Tagen.



In [146]:
idx = 90
translate(input_text=qs[:,0][idx],
          true_output_text=qs[:,1][idx])

C:\Users\Tammaa\anaconda3\envs\env1\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Input text:
ssss Mr Hänsch represented you on this occasion. eeee

Translated text:
 herr solana haben sie sich mit den                                                                                  

True output text:
Der Kollege Hänsch hat Sie dort vertreten.



In [144]:
translate(input_text="The term political statement is used to refer to any act or non-verbal form of communication that is intended to influence a decision to be made for or by a political party. A political statement can vary from a mass demonstration to the wearing of a badge with a political slogan",
          true_output_text='Ich möchte der Europäischen Union beitreten')

C:\Users\Tammaa\anaconda3\envs\env1\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Input text:
The term political statement is used to refer to any act or non-verbal form of communication that is intended to influence a decision to be made for or by a political party. A political statement can vary from a mass demonstration to the wearing of a badge with a political slogan

Translated text:
 deshalb ist eine politische entscheidung über eine politische entscheidung oder einer entscheidung zu einer entscheidung zu einer entscheidung zu einer entscheidung zu einer entscheidung zu einer entscheidung zu einer entscheidung zu einer entscheidung zu einer entscheidung zu einer entscheidung zu einer entscheidung zu einer entscheidung zu einer entscheidung

True output text:
Ich möchte der Europäischen Union beitreten



In [ ]:
qs = np.array(qs)

In [ ]:
qs[:,1]

In [ ]:
eng_tokenizer = tokenization(`)
de_tokenizer = tokenization(qs[:,1])

In [ ]:
eng_vocab_size = len(eng_tokenizer.word_index)+1
eng_vocab_size

In [ ]:
de_vocab_size = len(de_tokenizer.word_index)+1
de_vocab_size

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
max_encoder_seq_length =  max([len(x) for x in qs[:,0]])
max_decoder_seq_length = max([len(x) for x in qs[:,1]])

In [ ]:
print(max_encoder_seq_length, max_decoder_seq_length)

In [ ]:
def encode_sequences(tokenizer, length, lines):
    seq = tokenizer.texts_to_sequences(lines)
    seq = pad_sequences(seq,maxlen=length,padding='post')
    return seq

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(qs,test_size=0.2, random_state=12)

In [ ]:
train[:,1]

In [ ]:
en_length = max_encoder_seq_length
de_length = max_decoder_seq_length
units= 512

In [ ]:
train_x = encode_sequences(de_tokenizer,en_length, train[:,1])
train_y = encode_sequences(eng_tokenizer,de_length,train[:,0])

test_x = encode_sequences(de_tokenizer,en_length, test[:,1])
test_y = encode_sequences(eng_tokenizer,de_length,test[:,0])

# Define The model

In [ ]:
from tensorflow.keras.layers import *
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import *

In [ ]:
units=512

In [ ]:
print(eng_vocab_size)

In [ ]:
def define_model(in_vocab, out_vocab, in_timesteps, out_timesteps, units):
    model = Sequential()
    model.add(Embedding(32,units,input_length=in_timesteps, mask_zero=True))
    model.add(LSTM(units))
    model.add(RepeatVector(out_timesteps))
    model.add(LSTM(units, return_sequences=True))
    
    model.add(Dense(32,activation='softmax'))
    return model

In [ ]:
model = define_model(eng_vocab_size,de_vocab_size,en_length,de_length,units)
rms = RMSprop(learning_rate=0.001)
model.compile(optimizer=rms,loss='sparse_categorical_crossentropy')

In [ ]:
import tensorflow as tf

In [ ]:
history = model.fit(train_x,train_y.reshape((train_y.shape[0],train_y.shape[1],1)),
                   epochs=50,
                    batch_size=512
                    )

In [ ]:
tf.keras.utils.plot_model(model,to_file="translation.png",show_shapes=True,dpi=92)

In [ ]:
def get_word(n, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index ==n:
            return word
    return None

In [ ]:
preds = model.predict(test_x.reshape((test_x.shape[0],test_x.shape[1])))

In [ ]:
preds

In [ ]:
preds_text = []
for i in preds:
    temp = []
    for j in range(len(i)):
        t = get_word(i[j], eng_tokenizer)
        if j>0:
            if (t== get_word(i[j-1], eng_tokenizer) or (t==None)):
                temp.append('')
            else:
                temp.append(t)
        else:
            if(t==None):
                temp.append('')
            else:
                temp.append(t)
    preds_text.append(' '.join(temp))

In [ ]:
preds_text